In [3]:
# Import modules
import datetime

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import spiceypy

# Clear any previously loaded kernels
spiceypy.kclear()

# Load kernels via meta file
spiceypy.furnsh('kernel_meta.txt')
print("Meta Kernel Loaded:", spiceypy.ktotal("ALL"))

# Manually load kernels (in correct order)
spiceypy.furnsh('../kernels/lsk/naif0012.tls')  # Leap Seconds
spiceypy.furnsh('../kernels/spk/de432s.bsp')    # Ephemeris Data
spiceypy.furnsh('../kernels/pck/pck00010.tpc')  # Planet Constants

# Verify total loaded kernels
print("Total Kernels Loaded:", spiceypy.ktotal("ALL"))

# We want to compute miscellaneous positions w.r.t. the centre of
# the Sun for a certain time interval.
# First, we set an initial time in UTC.
init_time_utc = datetime.datetime(year=2000, month=1, day=1, \
                                  hour=0, minute=0, second=0)

# Add a number of days; you can play around with the datetime variables; but
# leave it as it is for the first try, since other computations and comments
# are based on this value.
delta_days = 10000
end_time_utc = init_time_utc + datetime.timedelta(days=delta_days)

# Convert the datetime objects now to strings
init_time_utc_str = init_time_utc.strftime('%Y-%m-%dT%H:%H:%S')
end_time_utc_str = end_time_utc.strftime('%Y-%m-%dT%H:%H:%S')

# Print the starting and end times
print('Init time in UTC: %s' % init_time_utc_str)
print('End time in UTC: %s\n' % end_time_utc_str)

# Convert to Ephemeris Time (ET) using the SPICE function utc2et
init_time_et = spiceypy.utc2et(init_time_utc_str)
end_time_et = spiceypy.utc2et(end_time_utc_str)

# Create a numpy array that covers a time interval in delta = 1 day step
time_interval_et = np.linspace(init_time_et, end_time_et, delta_days)

Meta Kernel Loaded: 1
Total Kernels Loaded: 4
Init time in UTC: 2000-01-01T00:00:00
End time in UTC: 2027-05-19T00:00:00



In [4]:
# Using km is not intuitive. AU would scale it too severely. Since we compute
# the Solar System Barycentre (SSB) w.r.t. the Sun; and since we expect it to
# be close to the Sun, we scale the x, y, z component w.r.t the radius of the
# Sun. We extract the Sun radii (x, y, z components of the Sun ellipsoid) and
# use the x component
_, radii_sun = spiceypy.bodvcd(bodyid=10, item='RADII', maxn=3)

radius_sun = radii_sun[0]

In [ ]:
solar_system_df = pd.DataFrame()

solar_system_df.loc[:, "ET"] = time_interval_et

solar_system_df.loc[:, "UTC"] = \
    solar_system_df["ET"].apply(lambda x: spiceypy.et2datetime(et=x).date())

solar_system_df.loc[:, "POS_SSB_WRT_SUN"] = \
    solar_system_df["ET"].apply(lambda x: spiceypy.spkgps(targ=0, 
                                                          et=x, ref='ECLIPJ2000', obs=10)[0])

solar_system_df.loc[:, "POS_SSB_WRT_SUN_SCALED"] = \
    solar_system_df["POS_SSB_WRT_SUN"].apply(lambda x: x / radius_sun)

solar_system_df.loc[:, "POS_SSB_WRT_SUN_SCALED"] = \
    solar_system_df["POS_SSB_WRT_SUN_SCALED"].apply(lambda x: spiceypy.vnorm(x))

                ET         UTC  \
0    -4.313582e+04  2000-01-01   
1     4.327283e+04  2000-01-02   
2     1.296815e+05  2000-01-03   
3     2.160901e+05  2000-01-04   
4     3.024987e+05  2000-01-05   
...            ...         ...   
9995  8.636112e+08  2027-05-14   
9996  8.636976e+08  2027-05-15   
9997  8.637841e+08  2027-05-16   
9998  8.638705e+08  2027-05-17   
9999  8.639569e+08  2027-05-19   

                                        POS_SSB_WRT_SUN POS_SSB_WRT_SUN_SCALED  
0     [1068000.1891061615, 417680.9418664607, -30844...               1.648254  
1     [1067195.5017360714, 418789.11029741366, -3083...               1.647758  
2     [1066389.5596805303, 419896.06924681185, -3081...               1.647262  
3     [1065582.365061001, 421001.82313176966, -30802...               1.646766  
4     [1064773.9198190796, 422106.3763607678, -30788...               1.646269  
...                                                 ...                    ...  
9995  [4663.265215005825